In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [2]:
import warnings
warnings.simplefilter("ignore")

In [6]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [7]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [8]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']


In [9]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [11]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

In [12]:
df_columns

['Data',
 '#Sample',
 '#Dimensions',
 'Outlier Perc',
 'PCA',
 'MCD',
 'OCSVM',
 'LOF',
 'CBLOF',
 'KNN',
 'HBOS',
 'ABOD',
 'IFOREST',
 'FEATUREBAGGING']

In [13]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


In [14]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n... Processing",mat_file,'.....')
    mat = loadmat(mat_file)
    
    X = mat['X']
    y = mat['y'].ravel()
    
    outliers_fraction = np.count_nonzero(y) / len(y)
    
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = random_state)
    
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    classifiers = {'Angle-based Outlier Detector (ABOD)' : ABOD(contamination=outliers_fraction), 
                  'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction,check_estimator=False ,random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                  'Historgram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                  'Isolation Forest' : IForest(contamination=outliers_fraction, random_state=random_state),
                  'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)' : LOF(contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                  'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)}
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, 4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), 4)
        prn = round(precision_n_scores(y_test, test_scores),4)
        
        print(f"{clf_name} ROC: {roc}, precision @ rank n: {prn}, execution time: {duration}s' ")
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)
    
    


... Processing arrhythmia.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7687, precision @ rank n: 0.3571, execution time: 2.3533s' 
Cluster-based Local Outlier Factor ROC: 0.7684, precision @ rank n: 0.4643, execution time: 2.3878s' 
Feature Bagging ROC: 0.7799, precision @ rank n: 0.5, execution time: 1.1473s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.8511, precision @ rank n: 0.5714, execution time: 2.8336s' 
Isolation Forest ROC: 0.8478, precision @ rank n: 0.5357, execution time: 0.723s' 
K Nearest Neighbors (KNN) ROC: 0.782, precision @ rank n: 0.5, execution time: 0.1783s' 
Local Outlier Factor (LOF) ROC: 0.7787, precision @ rank n: 0.4643, execution time: 0.1468s' 
Minimum Covariance Determinant (MCD) ROC: 0.8228, precision @ rank n: 0.4286, execution time: 1.0416s' 
One-class SVM (OCSVM) ROC: 0.7986, precision @ rank n: 0.5, execution time: 0.0937s' 
Principal Component Analysis (PCA) ROC: 0.7997, precision @ rank n: 0.5, execution time: 0.1425s' 

... Processing

In [15]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.3116,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2615,0.0462,0.0462,0.0615,0.2923,0.3385


In [16]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,2.3533,2.3878,1.1473,2.8336,0.723,0.1783,0.1468,1.0416,0.0937,0.1425
0,cardio,1831,21,9.6122,0.6798,0.2385,1.3927,0.008,0.7385,0.26,0.1765,0.8505,0.143,0.0102
0,glass,214,9,4.2056,0.0774,0.0571,0.0792,0.008,0.5036,0.016,0.0079,0.0718,0.0081,0
0,ionosphere,351,33,35.8974,0.1123,0.0663,0.1406,0.016,0.5566,0.024,0.008,0.104,0.008,0.016
0,letter,1600,32,6.25,0.6036,0.1897,1.2948,0.0155,0.7292,0.2268,0.1587,1.6158,0.1488,0.0239
0,lympho,148,18,4.0541,0.0639,0.0849,0.0803,0.0169,0.5415,0,0,0.063,0,0
0,mnist,7603,100,9.2069,13.7687,1.0906,95.2307,0.0947,3.2197,12.0709,11.5598,4.1017,7.8101,0.2512
0,musk,3062,166,3.1679,4.0848,0.3549,24.8751,0.0951,2.0772,3.3264,3.183,17.0967,2.2377,0.2917
0,optdigits,5216,64,2.8758,4.6925,0.6736,26.8587,0.0593,1.733,3.3851,3.1055,1.7864,2.3573,0.0781
0,pendigits,6870,16,2.2707,2.619,0.37,7.7421,0.016,1.2133,1.0169,0.972,3.2383,1.4067,0.016
